In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import *
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
data = pd.read_csv("../../../../datasets/market/top_coins/BTC.csv")
# Convert Date to datetime and sort values in ascending order
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%y')
data = data.sort_values(by='Date')

# Select only Date and Close columns for time series forecasting
data = data[['Date', 'Close']]
print("Overview of dataset \n\n" ,data.head() )

In [ ]:
# Extract Close price values
closing_prices = data['Close'].values.reshape(-1, 1)

# Normalize the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(closing_prices)

In [ ]:
# Split data into training and testing sets 
train_size = int(len(scaled_data) * 0.7)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

In [ ]:
# Prepare function to create sequences for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

In [ ]:
# Define sequence length
sequence_length = 60
# Create sequences for training and testing
X_train, y_train = create_sequences(train_data, sequence_length)
X_test, y_test = create_sequences(test_data, sequence_length)

In [ ]:
# Reshape input data for LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

X_train.shape, X_test.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
# build LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Train the model
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=50, batch_size=35, validation_data=(X_test, y_test), callbacks=[early_stop])

In [ ]:
# Evaluate model
train_loss = model.evaluate(X_train, y_train)
test_loss = model.evaluate(X_test, y_test)
print(f"Train Loss: {train_loss:.4f}")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
# Make predictions on test data
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
# Evaluate metrics
rmse = np.sqrt(mean_squared_error(actual_prices, predictions))
mae = mean_absolute_error(actual_prices, predictions)
r2 = r2_score(actual_prices, predictions)

# Print the metrics
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
# Plot predicted vs. actual prices with date labels
plt.figure(figsize=(10, 6))
# Create date range for test data
test_dates = data['Date'].iloc[-len(actual_prices):]  # Select dates for the test period
# Plot actual prices with dates
plt.plot(test_dates, actual_prices, color='blue', label='Actual BTC Price')
# Plot predicted prices with dates
plt.plot(test_dates, predictions, color='red', label='Predicted BTC Price')
# Customize the plot
plt.title('BTC Price Prediction with LSTM')
plt.xlabel('Date')
plt.ylabel('BTC Price')
plt.legend()
# Rotate and format the date labels for better readability
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Forecast the next 7 days
last_60_days = scaled_data[-60:].reshape(1, -1, 1)
future_prices = []
for _ in range(7):
    future_pred = model.predict(last_60_days)
    future_prices.append(future_pred[0, 0])
    
    # Correct reshaping to maintain 3D shape for LSTM
    future_pred_reshaped = future_pred.reshape(1, 1, 1)
    last_60_days = np.append(last_60_days[:, 1:, :], future_pred_reshaped, axis=1)

# Inverse transform future prices
future_prices = scaler.inverse_transform(np.array(future_prices).reshape(-1, 1))

# Plot future price prediction
future_dates = pd.date_range(data['Date'].iloc[-1] + pd.Timedelta(days=1), periods=7)
plt.figure(figsize=(10, 5))
plt.plot(future_dates, future_prices, marker='o', color='green', label='Forecasted Prices')
plt.title('BTC Price Forecast for Next 7 Days')
plt.xlabel('Date')
plt.ylabel('BTC Price')
plt.legend()
plt.show()

In [ ]:
# Define backtesting window (last 7 days of test data)
backtest_window = 7
# Create sequences for backtest
X_backtest = scaled_data[-(60 + backtest_window):-backtest_window].reshape(1, -1, 1)
# Forecast for the last 7 days using the model
backtest_forecast = []
for _ in range(backtest_window):
    pred = model.predict(X_backtest)
    backtest_forecast.append(pred[0, 0])
    X_backtest = np.append(X_backtest[:, 1:, :], pred.reshape(1, 1, 1), axis=1)
# Inverse transform the forecasted values
backtest_forecast = scaler.inverse_transform(np.array(backtest_forecast).reshape(-1, 1))
# Get actual prices for comparison
actual_future_prices = data['Close'].values[-backtest_window:]
# Calculate RMSE and MAE for the backtest
rmse_forecast = np.sqrt(mean_squared_error(actual_future_prices, backtest_forecast))
mae_forecast = mean_absolute_error(actual_future_prices, backtest_forecast)
print(f"Backtest RMSE: {rmse_forecast:.2f}")
print(f"Backtest MAE: {mae_forecast:.2f}")